# Guided Project: Building a Spam Filter with Naive Bayes

In this project, we will be building a spam filter using the multinomial Naive Bayes algorithm along with a dataset of 5,572 SMS messages that are already classified by humans. 

To classify messages as spam or non-spam, the computer will:

    1. Learns how humans classify messages.
    2. Uses that human knowledge to estimate probabilities for new messages — probabilities for spam and non-spam.
    3. Classifies a new message based on these probability values — if the probability for spam is greater, then it classifies the message as spam. Otherwise, it classifies it as non-spam (if the two probability values are equal, then we may need a human to classify the message).
 
The dataset to be used was put together by Tiago A. Almeida and José María Gómez Hidalgo, and it can be downloaded from the The UCI Machine Learning [Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection).

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('SMSSpamCollection', sep='\t',header = None, names = ['Label','SMS'])
df.shape

(5572, 2)

In [3]:
df.head()

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
Label    5572 non-null object
SMS      5572 non-null object
dtypes: object(2)
memory usage: 87.1+ KB


In [5]:
df['Label'].value_counts(normalize = True)

ham     0.865937
spam    0.134063
Name: Label, dtype: float64

86% of the data is non spam while the remaining 13% is spam.

### Data preparation by shuffling and splitting into training and test data

To calculate all the probabilities required by the algorithm, we'll first need to perform a bit of data cleaning to bring the data in a format that will allow us to extract easily all the information we need.

In [6]:
# Randomize the data
random_df = df.sample(frac=1, random_state=1)

In [7]:
# Calculate index for split
index = round(len(random_df) * 0.8)

In [8]:
#Split data into 80% training data and 20% test data
train_data = random_df[:index].reset_index(drop = True)
test_data = random_df[index:].reset_index(drop= True)

In [9]:
#Check how the splited data reflect the original
train_data['Label'].value_counts(normalize=True)

ham     0.86541
spam    0.13459
Name: Label, dtype: float64

In [10]:
test_data['Label'].value_counts(normalize=True)

ham     0.868043
spam    0.131957
Name: Label, dtype: float64

Thanks to shuffling the data, the test and training data both have spam and non-spam data split similar to the original dataset.

### Data cleaning and transformation

Before applying the algorithm we need to bring our training data to a form where probality rules can be applied. To do this, with the exception of the "Label" column, every other column in the transformed table should represent a unique word in our vocabulary. More specifically, each column shows the frequency of that unique word for any given message

In [11]:
# Remove special characters and make all text lower case
import re

train_data['SMS'] =train_data['SMS'].str.replace('\W', ' ')
train_data['SMS'] =train_data['SMS'].str.lower()

In [12]:
#convert column to list of strings
train_data['SMS'] = train_data['SMS'].str.split()


In [13]:
#Building a vocabulary of all the unique words in the training data
vocabulary = []
for messages in train_data['SMS']:
    for words in messages:
        vocabulary.append(words)
        
vocab_set = set(vocabulary)
vocab_list = list(vocab_set)

In [14]:
print('a total of {} unique words are in the training data'.format(len(vocab_set)))

a total of 7783 unique words are in the training data


In [15]:
# creating a dictionary of unique words and their counts per sms
word_counts_per_sms = {unique_word: [0] * len(train_data['SMS']) for unique_word in vocabulary}

for index, sms in enumerate(train_data['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1

In [16]:
# Transform the dictionary to dataframe
testing_df =pd.DataFrame(word_counts_per_sms)

In [17]:
testing_df.head(3)

,0,00,000,000pes,008704050406,0089,01223585334,02,0207,02072069400,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
test_df = pd.concat([train_data, testing_df], axis = 1)

In [19]:
test_df.head(3)

,Label,SMS,0,00,000,000pes,008704050406,0089,01223585334,02,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Calculating constants

Now we are set to start applying the spam filter algorithm, We can calculate the value of constants to be used in Naive Bayes once and avoid doing the computations again when a new messages comes in. Some of this constants will have the same value for every new message. 
Below, we'll use our training set to calculate:

    P(Spam) and P(Ham)
    NSpam, NHam, NVocabulary

We'll also use Laplace smoothing and set $\alpha = 1$.

In [20]:
# Seperate spam and ham rows into individual dataset
spam_data = test_df[test_df['Label']== 'spam']
ham_data = test_df[test_df['Label']== 'ham']

# probability of spam data  P(Spam)
p_spam = len(spam_data)/len(test_df)

# probability of ham data  P(Ham)
p_ham = len(ham_data)/len(test_df)

# No of words in spam messages (Nspam)
words_per_spam = spam_data['SMS'].apply(len)
n_spam = words_per_spam.sum()

# No of words in ham messages  (NHam)
words_per_ham = ham_data['SMS'].apply(len)
n_ham = words_per_ham.sum()

# No of words in the vocabulary
n_vocab = len(vocab_list)

#alpha
alpha = 1

### Calculation parameters

To make the algorithm work very fast on new message, we should calculate probabilities for all the unique words in the training data (Nvocabulary), then they won't have to be calculated everytime the algorithm is run. The parameters $P(w_i|Spam)$ and $P(w_i|Ham)$. Each parameter will thus be a conditional probability value associated with each word in the vocabulary.

In [21]:
# Initiate parameters
spam_parm = {unique_word:0 for unique_word in vocab_list}
ham_parm = {unique_word:0 for unique_word in vocab_list}


# Calculate parameters
for word in vocab_list:
    n_word_given_spam = spam_data[word].sum()   # spam_data already defined in a cell above
    p_word_given_spam = (n_word_given_spam + alpha) / (n_spam + alpha*n_vocab)
    spam_parm[word] = p_word_given_spam
    
    n_word_given_ham = ham_data[word].sum()   # ham_data already defined in a cell above
    p_word_given_ham = (n_word_given_ham + alpha) / (n_ham + alpha*n_vocab)
    ham_parm[word] = p_word_given_ham



### Creating function to classify new messages

Now that we have all our parameters calculated, we can start creating the spam filter. The spam filter can be understood as a function that:

    Takes in as input a new message (w1, w2, ..., wn).
    Calculates P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn).
    Compares the values of P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn), and:
        If P(Ham|w1, w2, ..., wn) > P(Spam|w1, w2, ..., wn), then the message is classified as ham.
        If P(Ham|w1, w2, ..., wn) < P(Spam|w1, w2, ..., wn), then the message is classified as spam.
        If P(Ham|w1, w2, ..., wn) = P(Spam|w1, w2, ..., wn), then the algorithm may request human help.


In [22]:
import re

def classify(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()
    
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    for word in message:
        if word in spam_parm:
            p_spam_given_message *= spam_parm[word]
            
        if word in ham_parm:
            p_ham_given_message *= ham_parm[word]
            

    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)

    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')

In [26]:
# Testing the function with spam
classify('WINNER!! This is the secret code to unlock the money: C3421.')

P(Spam|message): 1.3481290211300841e-25
P(Ham|message): 1.9368049028589875e-27
Label: Spam


In [27]:
# testing the function with ham
classify("Sounds good, Tom, then see u there")

P(Spam|message): 2.4372375665888117e-25
P(Ham|message): 3.687530435009238e-21
Label: Ham


### Testing function on test data

The algorithm will output a classification label for every message in our test set, which we'll be able to compare with the actual label (given by a human). Note that, in training, our algorithm didn't see these 1,114 messages, so every message in the test set is practically new from the perspective of the algorithm. We will also modify our function to return the labels spam or ham and not print them.

In [31]:
def classify_test_set(message):
    
    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()
    
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    for word in message:
        if word in spam_parm:
            p_spam_given_message *= spam_parm[word]
            
        if word in ham_parm:
            p_ham_given_message *= ham_parm[word]
            

    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_ham_given_message < p_spam_given_message:
        return 'spam'
    else:
        return 'needs human classification'

    

In [32]:
# Creating a new column for the labels returned
test_data['predicted'] = test_data['SMS'].apply(classify_test_set)
test_data.head()

,Label,SMS,predicted
0,ham,Later i guess. I needa do mcat study too.,ham
1,ham,But i haf enuff space got like 4 mb...,ham
2,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,ham,"All done, all handed in. Don't know if mega sh...",ham


In [40]:
correct = 0
total = len(test_data)

for row in test_data.iterrows():
    row = row[1]
    if row['Label'] == row['predicted']:
        correct += 1
        
accuracy = correct/total
print('The created algorithm is {:.2f}% accurate'.format(accuracy * 100))

The created algorithm is 98.74% accurate


### Conclusion

In this project, we managed to build a spam filter for SMS messages using the multinomial Naive Bayes algorithm. The filter had an accuracy of 98.74% on the test set, which is an excellent result. We initially aimed for an accuracy of over 80%, but we managed to do way better than that.